# Data preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('muted')
sns.set_style('whitegrid')
%matplotlib inline
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm, tqdm_notebook
#tqdm_notebook().pandas()
tqdm.pandas(tqdm_notebook)

## Medicare data set

This is the data preparation of the Medicare Drug Spending Part D dataset for the years 2011-2015.

### Read data, change labels and make year a variable

In [2]:
df = pd.read_excel('data/Medicare_Drug_Spending_PartD_All_Drugs_YTD_2015_12_06_2016.xlsx', 
                   sheetname='Data', skiprows=3)
id_cols = ['Brand Name', 'Generic Name']
cols_2011 = [col for col in df.columns if '2011' in col]
cols_2012 = [col for col in df.columns if '2012' in col]
cols_2013 = [col for col in df.columns if '2013' in col]
cols_2014 = [col for col in df.columns if '2014' in col]
cols_2015 = [col for col in df.columns if '2015' in col][:-1]
col_names = ['Brand Name', 'Generic Name', 'Claim Count',
       'Total Spending', 'Beneficiary Count',
       'Total Annual Spending Per User', 'Unit Count',
       'Average Cost Per Unit (Weighted)',
       'Beneficiary Count No LIS',
       'Average Beneficiary Cost Share No LIS',
       'Beneficiary Count LIS',
       'Average Beneficiary Cost Share LIS']
df1 = df[id_cols + cols_2011]; df1.columns=col_names
df2 = df[id_cols + cols_2012]; df2.columns=col_names
df3 = df[id_cols + cols_2013]; df3.columns=col_names
df4 = df[id_cols + cols_2014]; df4.columns=col_names
df5 = df[id_cols + cols_2015]; df5.columns=col_names# + ['Annual Change in Average Cost Per Unit, 2015']
df1['Year'] = 2011
df2['Year'] = 2012
df3['Year'] = 2013
df4['Year'] = 2014
df5['Year'] = 2015
df5.columns
data = df1.append([df2, df3, df4, df5])
data['Generic Name'] = data['Generic Name'].apply(lambda x: x.lower()[:-1])
data['Brand Name'] = data['Brand Name'].apply(lambda x: x.lower()[:-1])

/Users/dario/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dario/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dario/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [6]:
data.isnull().sum()

Brand Name                                  0
Generic Name                                0
Claim Count                              5188
Total Spending                           5188
Beneficiary Count                        6065
Total Annual Spending Per User           6065
Unit Count                               5188
Average Cost Per Unit (Weighted)         5188
Beneficiary Count No LIS                 8252
Average Beneficiary Cost Share No LIS    8484
Beneficiary Count LIS                    8252
Average Beneficiary Cost Share LIS       8398
Year                                        0
dtype: int64

Drug-disease pair keys

In [3]:
diseases = pd.read_table('data/drug-disease_TTD2016.txt', skiprows=11)
diseases['LNM'] = diseases['LNM'].apply(lambda x: x.lower())

In [11]:
def match_drugname(name):
    if name in diseases['LNM'].values:
        return name
    else:
        p = process.extractOne(name, choices=diseases['LNM'], scorer=fuzz.token_set_ratio, score_cutoff=90)
        if p:
            return p[0]
        else:
            return np.nan

def fuzzy_drugname(name):
    p = process.extractOne(name, choices=diseases['LNM'], scorer=fuzz.token_set_ratio, score_cutoff=90)
    if p:
        return p[0]
    else:
        return np.nan

In [17]:
process.extractOne('emtricitabine/tenofovir', choices=diseases['LNM'], scorer=fuzz.ratio)[0]

'emtricitabine'

In [39]:
diseases[diseases['LNM']=='emtricitabine']['Indication'].iloc[0]
'emtricitabine' in diseases['LNM'].values

True

In [ ]:
data['Matched Drug Name'] = data['Generic Name'].progress_apply(match_drugname)#.map(lambda x: diseases[diseases['LNM']==x]['Indication'].iloc[0])

In [ ]:
data['Indication'] = data['Matched Drug Name'].progress_apply(lambda x: diseases[diseases['LNM']==x]['Indication'].iloc[0] \
    if x in diseases['LNM'].values else np.nan)

In [ ]:
data.to_csv('medicare_data_disease.csv')
diseases.to_csv('drug-disease_keys.csv')

In [ ]:
bhvdata = pd.read_sas('data/LLCP2015.xpt')
bhvdata.MENTHLTH.hist()
bhvdata.ADDEPEV2.hist()
bhvdata.SXORIENT.hist()